In [10]:
import pandas as pd

# Let's read the newly uploaded CSV file to see what the data looks like
file_path_one_season = 'Result_Premier_2223.csv'
df = pd.read_csv(file_path_one_season)

# Display the first few rows of the DataFrame
df.head()

import numpy as np

# Function to extract home and away goals from the match result string "a~b"
def extract_goals(result):
    if pd.isna(result):
        return np.nan, np.nan
    home_goal, away_goal = map(int, result.split("~"))
    return home_goal, away_goal

# Initialize lists to store the goals
home_goals = []
away_goals = []

# Loop through each row and column to extract home and away goals
for i in range(df.shape[0]):
    for j in range(1, df.shape[1]):  # Starting from 1 to skip the "Home \ Away" column
        home_goal, away_goal = extract_goals(df.iloc[i, j])
        if np.isnan(home_goal) or np.isnan(away_goal):
            continue
        home_goals.append(home_goal)
        away_goals.append(away_goal)

# Calculate the overall mean number of goals (lambda_hat)
total_goals = home_goals + away_goals
lambda_hat = np.mean(total_goals)

# Calculate GBI using the formula
N = len(home_goals)  # Total number of matches
GBI = (1 / ((2 * N - 1) * lambda_hat)) * np.sum(np.square(np.array(total_goals) - lambda_hat))

lambda_hat, GBI




(1.4092105263157895, 1.1927323410699369)

In [30]:
import pandas as pd
import numpy as np

# Function to extract home and away goals from the match result string "a~b"
def extract_goals(result):
    if pd.isna(result):
        return np.nan, np.nan
    home_goal, away_goal = map(int, result.split("~"))
    return home_goal, away_goal

def compute_hicb_with_adjusted_mapping(df):
    # Initialize a dictionary to store points for each team
    points = {team: 0 for team in df['Home \\ Away']}
    
    # Create a mapping of full team names to abbreviations (adjusted for whitespace)
    team_mapping_adjusted = dict(zip(df['Home \\ Away'].str.strip(), [col.strip() for col in df.columns[1:]]))
    
    # Loop through each row and column to calculate points
    for i in range(df.shape[0]):
        home_team = df.iloc[i, 0].strip()
        for j in range(1, df.shape[1]):
            away_team_abbrev = df.columns[j].strip()
            away_team = [key for key, value in team_mapping_adjusted.items() if value == away_team_abbrev][0]
            
            home_goal, away_goal = extract_goals(df.iloc[i, j])
            
            # Check for NaN values and continue if found
            if np.isnan(home_goal) or np.isnan(away_goal):
                continue
            
            # Assign points based on match results
            if home_goal > away_goal:  # Home win
                points[home_team] += 3
            elif home_goal < away_goal:  # Away win
                points[away_team] += 3
            else:  # Draw
                points[home_team] += 1
                points[away_team] += 1
    
    # Calculate the total points in the league
    total_points = sum(points.values())
    
    # Calculate the HICB
    n = len(points)
    Pi = [p / total_points for p in points.values()]
    HICB = n * sum([p**2 for p in Pi])
    
    return HICB

# Read the CSV file
df = pd.read_csv('Result_Premier_2223.csv')

# Compute HICB using the data with the adjusted mapping
hicb_value = compute_hicb_with_adjusted_mapping(df)
print(hicb_value)


1.1110119055671446
